In [141]:
import pandas as pd
import json

In [142]:
rki_zahlen = pd.read_csv('/Users/constanzebayer/Downloads/test/data/COVID-19-Faelle_7-Tage-Inzidenz_Landkreise.csv')
rki_zahlen.head()

,Meldedatum,Landkreis_id,Bevoelkerung,Faelle_gesamt,Faelle_neu,Faelle_7-Tage,Inzidenz_7-Tage
0,2020-01-03,1001,91113,0,0,0,0.0
1,2020-01-04,1001,91113,0,0,0,0.0
2,2020-01-05,1001,91113,0,0,0,0.0
3,2020-01-06,1001,91113,0,0,0,0.0
4,2020-01-07,1001,91113,0,0,0,0.0


In [143]:
rki_selected_columns = rki_zahlen[['Meldedatum', 'Landkreis_id', 'Bevoelkerung']]
test = rki_selected_columns.sort_values('Meldedatum', ascending=False).groupby(['Landkreis_id', 'Bevoelkerung'], as_index=False).apply(lambda x: x.head(1))
test.reset_index(drop=True, inplace=True)
test



,Meldedatum,Landkreis_id,Bevoelkerung
0,2022-10-27,1001,91113
1,2022-10-27,1002,246243
2,2022-10-27,1003,216277
3,2022-10-27,1004,79496
4,2022-10-27,1051,133969
...,...,...,...
406,2022-10-27,16073,100969
407,2022-10-27,16074,82513
408,2022-10-27,16075,79030
409,2022-10-27,16076,96102


In [144]:
berlin_rows = test[(test['Landkreis_id'] >= 11001) & (test['Landkreis_id'] <= 11012)]
berlin_rows

,Meldedatum,Landkreis_id,Bevoelkerung
324,2022-10-27,11001,377735
325,2022-10-27,11002,280671
326,2022-10-27,11003,404795
327,2022-10-27,11004,317229
328,2022-10-27,11005,240542
329,2022-10-27,11006,292255
330,2022-10-27,11007,342285
331,2022-10-27,11008,320204
332,2022-10-27,11009,273188
333,2022-10-27,11010,274853


In [145]:
berlin = berlin_rows.groupby('Meldedatum').sum('Bevoelkerung')
berlin['Landkreis_id']=11000
berlin.reset_index(drop=True, inplace=True)
berlin

,Landkreis_id,Bevoelkerung
0,11000,3677472


In [147]:
berlin_rows

,Meldedatum,Landkreis_id,Bevoelkerung
324,2022-10-27,11001,377735
325,2022-10-27,11002,280671
326,2022-10-27,11003,404795
327,2022-10-27,11004,317229
328,2022-10-27,11005,240542
329,2022-10-27,11006,292255
330,2022-10-27,11007,342285
331,2022-10-27,11008,320204
332,2022-10-27,11009,273188
333,2022-10-27,11010,274853


In [148]:
diff_keys = set(test.Landkreis_id).difference(berlin_rows.Landkreis_id)
without_berlin = test[test['Landkreis_id'].isin(diff_keys)][['Landkreis_id', 'Bevoelkerung']]

In [149]:
with_berlin = pd.concat([without_berlin, berlin], ignore_index=True)
with_berlin

,Landkreis_id,Bevoelkerung
0,1001,91113
1,1002,246243
2,1003,216277
3,1004,79496
4,1051,133969
...,...,...
395,16074,82513
396,16075,79030
397,16076,96102
398,16077,87807


In [150]:
old_api_data = pd.read_json('/Users/constanzebayer/Documents/GitHub/corona-api/static/landkreise-einwohner.json')
old_api_data = old_api_data.transpose().reset_index()
old_api_data

,index,landkreis,landkreisTyp,einwohnerzahl,lat,long
0,1001,Flensburg,Stadt,89934,54.785119,9.432369
1,1002,Kiel,Stadt,246601,54.329759,10.130283
2,1003,Lübeck,Stadt,215846,53.875705,10.723051
3,1004,Neumünster,Stadt,79905,54.088042,9.983596
4,1051,Dithmarschen,Kreis,133251,54.133993,9.109677
...,...,...,...,...,...,...
395,16073,Saalfeld-Rudolstadt,Landkreis,102139,50.638271,11.311129
396,16074,Saale-Holzland-Kreis,Landkreis,82816,50.905491,11.735917
397,16075,Saale-Orla-Kreis,Landkreis,79632,50.582527,11.713913
398,16076,Greiz,Landkreis,96668,50.753331,12.084158


In [151]:
comparison = old_api_data.merge(with_berlin, left_on='index', right_on='Landkreis_id')
comparison

,index,landkreis,landkreisTyp,einwohnerzahl,lat,long,Landkreis_id,Bevoelkerung
0,1001,Flensburg,Stadt,89934,54.785119,9.432369,1001,91113
1,1002,Kiel,Stadt,246601,54.329759,10.130283,1002,246243
2,1003,Lübeck,Stadt,215846,53.875705,10.723051,1003,216277
3,1004,Neumünster,Stadt,79905,54.088042,9.983596,1004,79496
4,1051,Dithmarschen,Kreis,133251,54.133993,9.109677,1051,133969
...,...,...,...,...,...,...,...,...
395,16073,Saalfeld-Rudolstadt,Landkreis,102139,50.638271,11.311129,16073,100969
396,16074,Saale-Holzland-Kreis,Landkreis,82816,50.905491,11.735917,16074,82513
397,16075,Saale-Orla-Kreis,Landkreis,79632,50.582527,11.713913,16075,79030
398,16076,Greiz,Landkreis,96668,50.753331,12.084158,16076,96102


In [152]:
comparison.set_index('index', inplace=True)

In [153]:
comparison = comparison.rename(columns={'einwohnerzahl': 'einwohnerzahl_alt', 
                          'Bevoelkerung': 'einwohnerzahl'})

In [154]:
comparison = comparison[['landkreis', 'landkreisTyp', 'einwohnerzahl', 'lat', 'long']]
comparison = comparison.transpose()
comparison

index,1001,1002,1003,1004,1051,1053,1054,1055,1056,1057,...,16068,16069,16070,16071,16072,16073,16074,16075,16076,16077
landkreis,Flensburg,Kiel,Lübeck,Neumünster,Dithmarschen,Herzogtum Lauenburg,Nordfriesland,Ostholstein,Pinneberg,Plön,...,Sömmerda,Hildburghausen,Ilm-Kreis,Weimarer Land,Sonneberg,Saalfeld-Rudolstadt,Saale-Holzland-Kreis,Saale-Orla-Kreis,Greiz,Altenburger Land
landkreisTyp,Stadt,Stadt,Stadt,Stadt,Kreis,Kreis,Kreis,Kreis,Kreis,Kreis,...,Landkreis,Landkreis,Landkreis,Landkreis,Landkreis,Landkreis,Landkreis,Landkreis,Landkreis,Landkreis
einwohnerzahl,91113,246243,216277,79496,133969,200819,167560,202014,318326,129687,...,68717,62089,105367,82103,56504,100969,82513,79030,96102,87807
lat,54.785119,54.329759,53.875705,54.088042,54.133993,53.596434,54.611088,54.211782,53.723182,54.245729,...,51.156333,50.431086,50.736692,51.032264,50.413317,50.638271,50.905491,50.582527,50.753331,50.961288
long,9.432369,10.130283,10.723051,9.983596,9.109677,10.608184,9.064954,10.86662,9.737143,10.363199,...,11.156919,10.737739,10.968152,11.507425,11.133108,11.311129,11.735917,11.713913,12.084158,12.410718


In [155]:
new_file_json = comparison.to_json(orient='columns', force_ascii = False)
new_file_json

'{"1001":{"landkreis":"Flensburg","landkreisTyp":"Stadt","einwohnerzahl":91113,"lat":54.785119,"long":9.4323687},"1002":{"landkreis":"Kiel","landkreisTyp":"Stadt","einwohnerzahl":246243,"lat":54.329759,"long":10.130283},"1003":{"landkreis":"Lübeck","landkreisTyp":"Stadt","einwohnerzahl":216277,"lat":53.875705,"long":10.723051},"1004":{"landkreis":"Neumünster","landkreisTyp":"Stadt","einwohnerzahl":79496,"lat":54.088042,"long":9.9835962},"1051":{"landkreis":"Dithmarschen","landkreisTyp":"Kreis","einwohnerzahl":133969,"lat":54.133993,"long":9.1096773},"1053":{"landkreis":"Herzogtum Lauenburg","landkreisTyp":"Kreis","einwohnerzahl":200819,"lat":53.596434,"long":10.608184},"1054":{"landkreis":"Nordfriesland","landkreisTyp":"Kreis","einwohnerzahl":167560,"lat":54.611088,"long":9.0649543},"1055":{"landkreis":"Ostholstein","landkreisTyp":"Kreis","einwohnerzahl":202014,"lat":54.211782,"long":10.86662},"1056":{"landkreis":"Pinneberg","landkreisTyp":"Kreis","einwohnerzahl":318326,"lat":53.723182

In [159]:
new_file_json_formatted = json.loads(new_file_json)
new_file_json_formatted

{'1001': {'landkreis': 'Flensburg',
  'landkreisTyp': 'Stadt',
  'einwohnerzahl': 91113,
  'lat': 54.785119,
  'long': 9.4323687},
 '1002': {'landkreis': 'Kiel',
  'landkreisTyp': 'Stadt',
  'einwohnerzahl': 246243,
  'lat': 54.329759,
  'long': 10.130283},
 '1003': {'landkreis': 'Lübeck',
  'landkreisTyp': 'Stadt',
  'einwohnerzahl': 216277,
  'lat': 53.875705,
  'long': 10.723051},
 '1004': {'landkreis': 'Neumünster',
  'landkreisTyp': 'Stadt',
  'einwohnerzahl': 79496,
  'lat': 54.088042,
  'long': 9.9835962},
 '1051': {'landkreis': 'Dithmarschen',
  'landkreisTyp': 'Kreis',
  'einwohnerzahl': 133969,
  'lat': 54.133993,
  'long': 9.1096773},
 '1053': {'landkreis': 'Herzogtum Lauenburg',
  'landkreisTyp': 'Kreis',
  'einwohnerzahl': 200819,
  'lat': 53.596434,
  'long': 10.608184},
 '1054': {'landkreis': 'Nordfriesland',
  'landkreisTyp': 'Kreis',
  'einwohnerzahl': 167560,
  'lat': 54.611088,
  'long': 9.0649543},
 '1055': {'landkreis': 'Ostholstein',
  'landkreisTyp': 'Kreis',
  '

In [166]:
with open('landkreise-einwohner.json', 'w') as f:
    json.dump(new_file_json_formatted, f, indent=1, sort_keys=True, ensure_ascii=False)